In [3]:
import os
import sys
import utils
reload(utils)
from utils import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Using Theano backend.


In [4]:
current_dir = os.getcwd()
print current_dir
data_dir = current_dir + '/data/fastai/redux/'
print data_dir

/Users/trinakarmakar/anaconda2
/Users/trinakarmakar/anaconda2/data/fastai/redux/


In [5]:
batch_size = 64

In [6]:
from vgg16 import Vgg16

In [ ]:
%cd $data_dir
%mkdir valid
%mkdir results
%mkdir -p sample/train
%mkdir -p sample/test
%mkdir -p sample/valid
%mkdir -p sample/results
%mkdir -p test/unknown

In [ ]:
%cd $data_dir/train

g = glob('*.jpg')
shuff = np.random.permutation(g)
for i in range(2000): 
    os.rename(shuff[i], data_dir+'valid/'+shuff[i])

In [ ]:
from shutil import copyfile

In [ ]:
g = glob('*.jpg')
shuff = np.random.permutation(g)
for i in range(200): 
    os.rename(shuff[i], data_dir+'sample/train/'+shuff[i])

In [ ]:
g = glob('*.jpg')
shuff = np.random.permutation(g)
for i in range(50): 
    os.rename(shuff[i], data_dir+'sample/valid/'+shuff[i])

In [ ]:
%cd $data_dir/train
%mkdir cats
%mkdir dogs
%mv cat.*.jpg ./cats/
%mv dog.*.jpg ./dogs/

%cd $data_dir/sample/train
%mkdir cats
%mkdir dogs
%mv cat.*.jpg ./cats/
%mv dog.*.jpg ./dogs/

%cd $data_dir/sample/valid
%mkdir cats
%mkdir dogs
%mv cat.*.jpg ./cats/
%mv dog.*.jpg ./dogs/


In [17]:
%cd $data_dir/valid
%mkdir cats
%mkdir dogs
%mv cat.*.jpg ./cats/
%mv dog.*.jpg ./dogs/

/Users/trinakarmakar/anaconda2/data/fastai/redux/valid


In [6]:
%cd $data_dir/test
print os.getcwd()
%mkdir unknown
%mv *.jpg ./unknown/

/Users/trinakarmakar/anaconda2/data/fastai/redux/test
/Users/trinakarmakar/anaconda2/data/fastai/redux/test


In [9]:
train_path = data_dir+'train/'
test_path = data_dir + 'test/'
result_path = data_dir + 'results/'
valid_path = data_dir + 'valid/'

In [16]:
print valid_path

/Users/trinakarmakar/anaconda2/data/fastai/redux/valid/


In [7]:
vgg = Vgg16()

In [14]:
batch_size=64
nb_epoch = 1
batches = vgg.get_batches(train_path, batch_size=batch_size)
vgg.finetune(batches)

Found 22750 images belonging to 2 classes.


In [18]:
val_batches = vgg.get_batches(valid_path, batch_size=batch_size*2)

Found 2000 images belonging to 2 classes.


In [19]:
#Not sure if we set this for all fits
vgg.model.optimizer.lr = 0.01

In [1]:
#we will be fitting our data against train and validate against valid set and store our weights
latest_weight_file = None
for epoch in range(nb_epoch):
    print "running epoch %d" % epoch
    vgg.fit(batches, val_batches, nb_epoch=1)
    latest_weight_file = "ft%d.h5" % epoch
    vgg.model.save_weights(result_path, latest_weight_file)
print("Completed %s fit operations" % nb_epoch)

NameError: name 'nb_epoch' is not defined

In [ ]:
#generate prediction
batches, preds = vgg.test(test_path, batch_size=batch_size*2)

Found 12500 images belonging to 1 classes.


In [ ]:
#for each image vgg.test generates two probabilities baed on how we ordered cats and dogs,
preds[:5]
filenames = batches.filenames
print filenames[:5]

In [ ]:
#We can verify column ordering by visualizing some images
from PIL import Image
Image(test_path+filenames[2])

In [ ]:
#save your test results in arrayas so that we can use it later
save_array(results_path+'test_pred.dat', preds)
save_array(results_path+'filenames.dat', filenames)

In [ ]:
#Validate predictions
vgg.load_models(results_path+latest_weight_file)

In [ ]:
val_batches, val_proba = vgg.test(vali_path, batch_size=batch_size)

In [ ]:
filenames = val_batch.filenames
expected_labels = val_batches.classes #0 or 1
#round our predictions to 0 or 1
our_predictions = val_proba[:,0]
our_labels = np.round(1-ourp_predictions)

In [ ]:
#plots is the helper function in the utils to let us plot images in our validation set
from keras.processing import image
def plots_index(idx, title=None):
    plots(image.loadimg(valid_path+filenames[i] for i in idx, titles = titles)
          
#Number of images to view
n_view = 4

In [ ]:
#1 A few correct labales
correct_labels = np.where(our_labels == expected_lables)[0]
print "Found %d correct labels" % len(correct_labels)
idx = np.permutation(correct_labels)[:n_view]
plots_index(idx, our_predictions[idx])

In [ ]:
#2 Few incorrect labels
incorrect_labels = np.where(our_labels != expected_lables)
print "found %d incorrect labels" % len(incorrect_labels)
idx = np.permutations(incorrect_labels)[:n_view]
plots_index(idx, ur_predictions[idx])

In [ ]:
#3a Images we are most confident as cats are actully cats
correct_cats = np.where((our_labels == 0) & (our_labels == expected_labels))[0]
print "Found %d confident cats lables" % len(correct_cats)
most_correct_cats = np.argsort(our_predictions[correct_cats])[::-1][:n_vew]
plots_index(correct_cats[most_correct_cats], our_predictions[correct_cats][most_correct_cats])

In [ ]:
#3b Images we are most confident as dogs are actually dogs
correct_dogs = n.where((our_labels == 1) & (our_labels == expected_labels))[0]
print "Found %d of correct dogs" % len(correct_dogs)
most_correct_dogs = np.argsort(our_predictions[correct_cats])[::-1][:n_view]
plots_index(correct_cats[most_correct_cats], our_predictions[correct_cats][most_correct_cats])

In [ ]:
#3c Images we are most confident as cats are actully dogs
incorrect_cats = np.where((our_labels == 0) & (our_labels != expected_labels))[0]
print "Found %d incorrect cats lables" % len(incorrect_cats)
if len(incorrect_cats):
    most_incorrect_cats = np.argsort(our_predictions[incorrect_cats])[::-1][:n_vew]
    plots_index(incorrect_cats[most_incorrect_cats], our_predictions[incorrect_cats][most_incorrect_cats])

In [ ]:
#3d Images we are most confident as cats are actully dogs
incorrect_dogs = np.where((our_labels == 1) & (our_labels != expected_labels))[0]
print "Found %d incorrect dogs lables" % len(incorrect_dogs)
if len(incorrect_dogs):
    most_incorrect_dogs = np.argsort(our_predictions[incorrect_dogs])[::-1][:n_vew]
    plots_index(incorrect_cats[most_incorrect_dogs], our_predictions[incorrect_dogs][most_incorrect_dogs])

In [ ]:
#Most incorrect predictions (i.e. whose proba is closest to 0.5)
most_uncertain = np.argsort(np.abs(our_predictions - 0.5))
plots_index(most_uncertain[:n_view], our_predictions[most_uncertain])

In [ ]:
#Most common way to analyze classification error is confusion matrix
from sklearn.matrix import confusion_matrix
cm = confusion_matrix(expected_labels, our_labels)
plot_confusion_matrix(cm, val_batches.class_indices)

In [ ]:
#Prepare for the kaggle submission
#Load our saved array
preds = load_array(results_path+'test_preds.dat')
filenames = load_array(results_path+'filenames.dat')

In [ ]:
#grab the dog predictions file
isdog = preds[:,1]
print "Raw Predictions:" + str(isdog[:5])
print "Mid Predictions:" + str(isdog[(isdog < 0.6) & (isdog > 0.4)])
print "Edge Predictions:" + str(isdog[(isdog  == 1) & (isdog == 0)])

In [ ]:
#Visualize log loss
from sklearn.metrics import log_loss
x = [i * .0001 for i in range(1,10000)]
y = [log_loss([1], [[i*.0001,1-(i*.0001)]],eps=1e-15) for i in range(1,10000,1)])]
plt.plot(x,y)
plt.axis([-.05, 1.1, -.8, 10])
plt.title("Log Loss when true label = 1")
plt.xlabel("predicted probability")
plt.ylabel("log loss")

plt.show()

In [ ]:
#Play a trick swap all 0s with 0.05 and all ones with 0.95 for our edge predctions
isdog = isdog.clip(min=0.05, max=0.95)

In [ ]:
#Extract image IDs from our filenames into test/unknown directory
filenames = batches.filenames
ids = np.array([int(f[8:f.find('.')]) for f in filenames])

In [ ]:
#Here we we join two variable into an array [ImageID, isdog]
subm = np.stack([ids, isdog], axis=1)
subm[:5]

In [ ]:
%cd $data_dir
submission_file_name = 'submission1.csv'
np.savetxt(submission_file_name, subm, fmt = '%d,%0.5f', header='id,label', comments='')
from IPython.display import FileLink
%cd $data_dir
FileLink(submission_file_name)